📢 [프롬프트 엔지니어링, LLM(ChatGPT) 기반의 AI 서비스 개발](https://fastcampus.co.kr/data_red_golbin) 강의에서 사용하는 예제입니다.

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 모델 로딩

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

In [3]:
peft_model_id = "golbin/koalpaca-polyglot-12.8b-bill"

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

model = PeftModel.from_pretrained(model, peft_model_id)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=5120, out_features=15360, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features

## 모델 추론 함수

In [6]:
from transformers.generation import GenerationConfig

gen_cfg = GenerationConfig.from_model_config(model.config)
gen_cfg.pad_token_id = tokenizer.eos_token_id

def gen(x):
    gen_cfg.temperature = 0.3
    gen_cfg.max_new_tokens = 128

    gened = model.generate(
        **tokenizer(
            f"User: {x}\n\nAssistant:",
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        do_sample=True,
        generation_config=gen_cfg
    )

    result = tokenizer.decode(gened[0])
    answer = result.split('Assistant:')

    return answer[1].strip()

In [7]:
gen('1980년대에 첫 번째 혁신적인 기술을 보셨다고 하셨는데, 어떤 기술이었나요?')

'1980년대에는 그래픽 사용자 인터페이스를 처음 보았습니다. 이는 현대의 운영 체제를 포함한 모든 운영 체제의 전신이었습니다. 운영 체제는 운영 체제가 어떻게 운영되는지에 대한 기본적인 개념을 포함한 모든 운영 체제의 전신이었습니다. 운영 체제는 운영 체제가 어떻게 운영되는지에 대한 기본적인 개념을 포함한 모든 운영 체제의 전신이었습니다. 운영 체제는 운영 체제가 어떻게 운영되는지에 대한 기본적인 개념을 포함한 모든 운영 체제의 전신이었습니다. 운영 체제는 운영 체제가 어떻게 운영되는지에'

## API 서버 구성

In [14]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install fastapi pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=e6541edfa8aa71c99de0743e6f2299ec3f3d967bb2fadb6e00f516fe6f2e6d26
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [15]:
!ngrok authtoken "1iQNo8tA8oyiZpTcKhcvhfthOVA_33uiwzT86kiiZDu7y4iDX"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [16]:
from fastapi import FastAPI
from pydantic import BaseModel

class Prompt(BaseModel):
  content: str

class Answer(BaseModel):
  content: str

app = FastAPI()

@app.get('/')
async def chat():
    return 'Hello World!'

@app.post('/chat')
async def chat(prompt: Prompt) -> Answer:
    answer = gen(prompt.content)
    return {'content': answer }

## API 서버 실행

In [17]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)

print('Public URL:', ngrok_tunnel.public_url)

nest_asyncio.apply()
uvicorn.run(app, port=8000)

Exception in thread Thread-12 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 146, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)
INFO:     Started server process [3000]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://ddff-34-123-140-81.ngrok-free.app
INFO:     1.234.201.248:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     1.234.201.248:0 - "GET /apple-touch-icon.png HTTP/1.1" 404 Not Found
INFO:     1.234.201.248:0 - "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 Not Found
INFO:     1.234.201.248:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     1.234.201.248:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     1.234.201.248:0 - "POST /chat HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [3000]
